In [10]:
import pandas as pd
import glob
import os
import torch
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.utils import shuffle
import sdv
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

In [6]:
data = pd.read_csv("data/train_X.csv")
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.003922,0.776471,0.584314,0.639216,0.564706,0.796078,0.0,1.0,6.983670e-08,9.632649e-08,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,0.003922,0.776471,0.713725,0.360784,0.184314,0.286275,0.0,1.0,6.983670e-08,9.632649e-08,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,0.003922,0.776471,0.525490,0.615686,0.882353,0.678431,0.0,1.0,6.983670e-08,9.632649e-08,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.003922,0.776471,0.568627,0.568627,0.545098,0.411765,0.0,1.0,6.983670e-08,9.632649e-08,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,0.003922,0.776471,0.807843,0.164706,0.890196,0.874510,0.0,1.0,6.983670e-08,9.632649e-08,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9813921,0.003922,0.776471,0.152941,0.000000,0.039216,0.141176,0.0,1.0,6.983670e-08,9.632649e-08,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
9813922,0.003922,0.776471,0.415686,0.647059,0.529412,0.643137,0.0,1.0,6.983670e-08,9.632649e-08,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
9813923,0.627451,0.207843,0.752941,0.658824,0.392157,0.403922,0.0,0.0,6.983670e-08,1.348571e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9813924,0.003922,0.776471,0.486275,0.945098,0.956863,0.784314,0.0,1.0,6.983670e-08,9.632649e-08,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [8]:
display(dir(sdv.single_table))

['CTGANSynthesizer',
 'CopulaGANSynthesizer',
 'GaussianCopulaSynthesizer',
 'TVAESynthesizer',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'base',
 'copulagan',
 'copulas',
 'ctgan',
 'utils']

In [11]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data)
python_dict = metadata.to_dict()

In [ ]:
metadata.validate_data(data=data)
import torch
print(torch.cuda.is_available()) 
print(torch.cuda.get_device_name(0)) 


True
NVIDIA GeForce RTX 4090


In [ ]:
synthesizer = CTGANSynthesizer(
    metadata,
    enforce_rounding=False,
    epochs=20,
    verbose=True
)

synthesizer.fit(data)

synthetic_data = synthesizer.sample(num_rows=1200000)

c:\Users\NAKULR\Documents\GitHub\IoT23-Anomaly-Detection\venv\Lib\site-packages\sdv\single_table\base.py:163: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

Gen. (-0.27) | Discrim. (0.10): 100%|██████████| 1/1 [05:20<00:00, 320.61s/it]


In [24]:
synthetic_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.003788,0.773117,0.877445,0.040193,0.395790,0.616338,0.000000,0.999711,0.000000,0.000169,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.003709,0.776581,0.229281,0.240585,0.064195,0.207123,0.000000,0.999681,0.000263,0.000085,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.003763,0.776592,0.258195,0.971332,0.302669,0.981555,0.000000,0.999811,0.000000,0.000208,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,0.004120,0.776954,0.715319,0.983933,0.084369,0.034429,0.000000,0.999960,0.000000,0.000140,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,0.004081,0.772852,0.210837,0.000000,0.375200,0.823546,0.000000,0.999515,0.000000,0.000030,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
5,0.004158,0.777665,0.558428,0.129183,0.543025,0.396289,0.000000,1.000000,0.000251,0.000132,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
6,0.003696,0.776176,0.047766,0.901047,0.779498,0.548465,0.000000,0.999950,0.000006,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
7,0.004091,0.776811,0.091710,0.028348,0.680908,0.539471,0.000000,1.000000,0.000261,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
8,0.392698,0.404960,0.874011,0.047780,0.052633,0.998787,0.999741,0.000325,0.000030,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
9,0.392091,0.772436,0.384179,0.389804,0.630809,0.619357,0.999639,0.000505,0.000000,0.000170,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
